In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver;


In [0]:
%sql
USE CATALOG workspace;
CREATE SCHEMA IF NOT EXISTS silver;
USE SCHEMA silver;


In [0]:

from pyspark.sql import functions as F

raw = spark.read.option("header", True).csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv"
)

silver_df = (
    raw
    .withColumn("event_time", F.to_timestamp("event_time"))
    .withColumn("price", F.col("price").cast("double"))
    .filter(F.col("price").isNotNull())
    .withColumn("event_date", F.to_date("event_time"))
)

silver_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.events")


In [0]:
%sql
DESCRIBE TABLE silver.events;


In [0]:
spark.sql("""
SELECT *
FROM silver.events
WHERE event_type = 'purchase'
""").explain(True)


In [0]:
%sql
CREATE TABLE silver.events_part
USING DELTA
PARTITIONED BY (event_date, event_type)
AS
SELECT *
FROM silver.events;


In [0]:
spark.sql("""
SELECT *
FROM silver.events_part
WHERE event_type = 'purchase'
AND event_date = '2019-10-10'
""").explain(True)



In [0]:
%sql
SHOW PARTITIONS silver.events_part;


In [0]:
%sql
OPTIMIZE silver.events_part
ZORDER BY (user_id, product_id);


In [0]:
%sql
DESCRIBE HISTORY silver.events_part;


In [0]:
import time

start = time.time()
spark.sql("""
SELECT *
FROM silver.events
WHERE user_id = '12345'
""").count()

print(f"Unoptimized time: {time.time() - start:.2f} sec")


In [0]:
start = time.time()
spark.sql("""
SELECT *
FROM silver.events_part
WHERE user_id = '12345'
""").count()

print(f"Optimized time: {time.time() - start:.2f} sec")


In [0]:
df = spark.table("silver.events_part")
df.count()   # just materialize, no caching
